In [ ]:
pip install pytorch-tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#ver1_train_h1n1 = pd.read_csv('/content/drive/MyDrive/flushot-data/ver1/train_h1n1_ver1.csv', header=None).iloc[1:,1:].reset_index(drop=True)
#ver1_test_h1n1 = pd.read_csv('/content/drive/MyDrive/flushot-data/ver1/test_h1n1_ver1.csv', header=None).iloc[1:,1:].reset_index(drop=True)

#train = pd.read_csv('/content/drive/MyDrive/flushot-data/census-income.data', header=None)

train_h1n1 = pd.read_csv('/content/drive/MyDrive/flushot-data/train_h1n1_no_encoding.csv', header=None).iloc[1:,1:].reset_index(drop=True)
test_h1n1 = pd.read_csv('/content/drive/MyDrive/flushot-data/test_h1n1_no_encoding.csv', header=None).iloc[1:,1:].reset_index(drop=True)


train_seasonal = pd.read_csv('/content/drive/MyDrive/flushot-data/train_seasonal_no_encoding.csv', header=None).iloc[1:,1:].reset_index(drop=True)
test_seasonal = pd.read_csv('/content/drive/MyDrive/flushot-data/test_seasonal_no_encoding.csv', header=None).iloc[1:,1:].reset_index(drop=True)

index_df = pd.read_csv('/content/drive/MyDrive/flushot-data/test_h1n1_no_encoding.csv')['respondent_id']

# index drop은 안함
#train_h1n1 = pd.read_csv('/content/drive/MyDrive/flushot-data/train_h1n1_no_encoding2.csv', header=None).iloc[1:,1:].reset_index(drop=True)
#train_seasonal = pd.read_csv('/content/drive/MyDrive/flushot-data/train_seasonal_no_encoding2.csv', header=None).iloc[1:,1:].reset_index(drop=True)

# h1n1 vaccine prediction

In [ ]:
train = train_h1n1
target = 23  # 23번째 컬럼
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index

test_h1n1[23] = '100'
test_h1n1["Set"] = 'test'
train = pd.concat([train, test_h1n1], axis = 0).reset_index(drop=True)

test_indices = train[train.Set=="test"].index

In [ ]:
nunique = train.nunique()
types = train.dtypes
print(nunique)
categorical_columns = []
categorical_dims =  {}
for col in train.columns:
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = train[col].fillna("-")
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train.fillna(train.loc[train_indices, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
unused_feat = ['Set']
features = [ col for col in train.columns if col not in unused_feat+[target]] 
cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]


X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices]

1       4
2       3
3       2
4       2
5       2
6       2
7       2
8       2
9       2
10      2
11      2
12      2
13      5
14      5
15      5
16      5
17      4
18      2
19     10
20      3
21      4
22      4
23      3
Set     3
dtype: int64
1 4
2 3
3 2
4 2
5 2
6 2
7 2
8 2
9 2
10 2
11 2
12 2
13 5
14 5
15 5
16 5
17 4
18 2
19 10
20 3
21 4
22 4
23 3
Set 3


In [ ]:
print(cat_idxs)
print(cat_dims)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
[5, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 6, 6, 5, 4, 2, 10, 3, 5, 5]


In [ ]:
print(X_train)
print(X_train.shape)

[[2 1 1 ... 2 1 1]
 [2 2 1 ... 1 1 1]
 [3 2 1 ... 0 2 1]
 ...
 [2 3 1 ... 1 2 1]
 [3 3 1 ... 0 1 1]
 [1 1 1 ... 1 2 1]]
(19900, 22)


In [ ]:
y_train.shape

(19900,)

In [ ]:
print(X_test)
print(X_test.shape)

[[3 3 1 ... 0 2 1]
 [2 2 1 ... 2 4 1]
 [3 3 1 ... 2 2 1]
 ...
 [1 2 1 ... 0 2 1]
 [4 2 1 ... 0 2 1]
 [3 2 1 ... 1 1 1]]
(26708, 22)


In [ ]:
clf = TabNetClassifier(
                      cat_idxs=cat_idxs,
                      cat_dims=cat_dims,
                      # define your embedding sizes : here just a random choice
                      cat_emb_dim=1,
                      optimizer_fn=torch.optim.Adam,
                      optimizer_params=dict(lr=2e-2),
                      scheduler_params={"step_size":50,
                                        "gamma":0.9},
                      scheduler_fn=torch.optim.lr_scheduler.StepLR,
                      mask_type='sparsemax',
                      n_steps = 2
                      )

Device used : cpu


In [ ]:
max_epochs = 50

history1 = clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=max_epochs , patience=20,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False,
)

epoch 0  | loss: 0.70464 | train_auc: 0.64263 | valid_auc: 0.63371 |  0:00:01s
epoch 1  | loss: 0.6317  | train_auc: 0.71178 | valid_auc: 0.71519 |  0:00:02s
epoch 2  | loss: 0.60214 | train_auc: 0.71983 | valid_auc: 0.72029 |  0:00:03s
epoch 3  | loss: 0.58993 | train_auc: 0.75232 | valid_auc: 0.75623 |  0:00:04s
epoch 4  | loss: 0.57525 | train_auc: 0.76245 | valid_auc: 0.76396 |  0:00:05s
epoch 5  | loss: 0.57394 | train_auc: 0.76487 | valid_auc: 0.76998 |  0:00:06s
epoch 6  | loss: 0.57286 | train_auc: 0.76914 | valid_auc: 0.77117 |  0:00:07s
epoch 7  | loss: 0.56129 | train_auc: 0.7735  | valid_auc: 0.77739 |  0:00:08s
epoch 8  | loss: 0.56792 | train_auc: 0.77597 | valid_auc: 0.77967 |  0:00:09s
epoch 9  | loss: 0.56327 | train_auc: 0.7789  | valid_auc: 0.78031 |  0:00:11s
epoch 10 | loss: 0.55619 | train_auc: 0.78191 | valid_auc: 0.78401 |  0:00:12s
epoch 11 | loss: 0.56609 | train_auc: 0.7815  | valid_auc: 0.78428 |  0:00:13s
epoch 12 | loss: 0.55907 | train_auc: 0.78555 | vali

In [ ]:
preds1 = clf.predict_proba(X_test)

In [ ]:
preds1[:,1]

array([0.4662373 , 0.2699048 , 0.75180584, ..., 0.19190265, 0.1861991 ,
       0.40661845], dtype=float32)

In [ ]:
import matplotlib.pyplot as plt
history_dict = history1
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

TypeError: ignored

# seasonal vaccien prediction

In [ ]:
train = train_seasonal

target = 21  # 21번째 컬럼
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index

test_seasonal[21] = '100'
test_seasonal["Set"] = 'test'
train = pd.concat([train, test_seasonal], axis = 0).reset_index(drop=True)

test_indices = train[train.Set=="test"].index

In [ ]:
nunique = train.nunique()
types = train.dtypes
print(nunique)
categorical_columns = []
categorical_dims =  {}
for col in train.columns:
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = train[col].fillna("-")
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train.fillna(train.loc[train_indices, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
unused_feat = ['Set']
features = [ col for col in train.columns if col not in unused_feat+[target]] 
cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]


X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices]

In [ ]:
clf = TabNetClassifier(
                      cat_idxs=cat_idxs,
                      cat_dims=cat_dims,
                      # define your embedding sizes : here just a random choice
                      cat_emb_dim=1,
                      optimizer_fn=torch.optim.Adam,
                      optimizer_params=dict(lr=2e-2),
                      scheduler_params={"step_size":50,
                                        "gamma":0.9},
                      scheduler_fn=torch.optim.lr_scheduler.StepLR,
                      mask_type='sparsemax',
                      n_steps = 2
                       
                      )

In [ ]:
max_epochs = 50

clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=max_epochs , patience=20,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False,
)

In [ ]:
preds2 = clf.predict_proba(X_test)

In [ ]:
preds2[:,1]

In [ ]:
submit = pd.DataFrame({'respondent_id': index_df,
                       'h1n1_vaccine': preds1[:,1],
                       'seasonal_vaccine': preds2[:,1] })

In [ ]:
# 2e-2 -> 0.7727
submit.to_csv('/content/drive/MyDrive/flushot-data/tabnet_result_epoch15.csv', index = False)

In [ ]:
# 2e-2 -> 0.7896
submit.to_csv('/content/drive/MyDrive/flushot-data/tabnet_result_epoch30.csv', index = False)

In [ ]:
# 2e-2 -> 0.7885
submit.to_csv('/content/drive/MyDrive/flushot-data/tabnet_result_epoch50.csv', index = False)

In [ ]:
# 2e-2 -> 0.79080
submit.to_csv('/content/drive/MyDrive/flushot-data/tabnet_result_epoch50_step2.csv', index = False)

In [ ]:
# 2e-2 -> 0.7842
submit.to_csv('/content/drive/MyDrive/flushot-data/tabnet_result_epoch50_step2_nd16.csv', index = False)

In [ ]:
# 2e-2 -> 0.7804
submit.to_csv('/content/drive/MyDrive/flushot-data/tabnet_result_epoch50_step2_nd16_na16.csv', index = False)

In [ ]:
# 2e-2 -> 0.7889
submit.to_csv('/content/drive/MyDrive/flushot-data/tabnet_result_epoch50_step1.csv', index = False)

In [ ]:
# 2e-2 -> / epoch 500 / pat = 20 => 7814
submit.to_csv('/content/drive/MyDrive/flushot-data/tabnet_result_epoch500.csv', index = False)

In [ ]:
# 2e-2 / epoch 100  / pat = 20 => 0.7814
submit.to_csv('/content/drive/MyDrive/flushot-data/tabnet_result_epoch100.csv', index = False)

In [ ]:
# 2e-3 / epoch 500  => 0.7770
submit.to_csv('/content/drive/MyDrive/flushot-data/tabnet_result_epoch500.csv', index = False)

In [ ]:
# 2e-3 / without_index_drop / epoch 100 => 0.7730
submit.to_csv('/content/drive/MyDrive/flushot-data/tabnet_result_epoch100_no_drop.csv', index = False)